<br><br>

## **Import necessary Python libraries and modules**

First, we will import necessary Python libraries and modules. These include as `gdown`, for downloading large files from Google Drive (where we will get our UCSD Goodreads reviews), as well as scikit-learn (`sklearn`) and PyTorch (`torch`), for various machine learning tools.

In [21]:

# For data manipulation and analysis
import pandas as pd
import numpy as np

# For machine learning tools and evaluation
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report


# For deep learning
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
import torch
#import transformers
from sklearn.metrics import confusion_matrix


To use the HuggingFace [`transformers` Python library](https://huggingface.co/transformers/installation.html), we will install it with `pip`.

In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 12.5 MB/s 
     |████████████████████████████████| 596 kB 40.7 MB/s 
     |████████████████████████████████| 6.5 MB 39.8 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
     |████████████████████████████████| 895 kB 49.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Once `transformers` is installed, we will import modules for `DistilBert`, a *distilled* or smaller version of a BERT model that runs more quickly and uses less computing power. This makes it ideal for those just getting started with BERT.

In [22]:
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizerFast
from transformers import Trainer, TrainingArguments

<br><br>

## **Set parameters and file paths**

In [23]:
# This is the name of the BERT model that we want to use. 
# We're using DistilBERT to save space (it's a distilled version of the full BERT model), 
# and we're going to use the cased (vs uncased) version.
model_name = 'xlm-roberta-base'  

# This is the name of the program management system for NVIDIA GPUs. We're going to send our code here.
device_name = 'cuda'       

# This is the maximum number of tokens in any document sent to BERT.
max_length = 128                                                        

# This is the name of the directory where we'll save our model. You can name it whatever you want.
#cached_model_directory_name = 'ABSA_FineTuning_BERT'  
cached_model_directory_name = 'Emotions_fin_xlmr'  

In [5]:
#stiamo utlizzando la GPU?
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [24]:
train=pd.read_csv("C:\\Users\\Fossati\\Desktop\\Tesi\\Dati\\Dati_FineTuningBERT\\Train_Test_emotion\\Strategy_fin\\train_def.csv")
train

,Unnamed: 0,level_0,Sentence,Emotions
0,0,39939,i have to admit that i feel a little irate as ...,Anger
1,1,27439,"[NAME] are the liars, we know.",Anger
2,2,30643,"$42,000 fine over a JOKE is unreasonable.",Anger
3,3,45392,i have that feeling but idol anime is pretty p...,Joy
4,4,8480,> To say NDP foreign policy consistently suppo...,Neutral
...,...,...,...,...
80662,80662,73600,최고의 영화. 다시 나타날 수 없는 작품,Sadness
80663,80663,46112,청춘이네 보는 내내 두근거렸다,Fear
80664,80664,97318,2시간짜리반전캠페인영화. 나에게서 캐산의 추억을 앗아갔다...,Surprise
80665,80665,83289,아 넘 질질 끌어요속 터져 못보겠습니다 빠른 전계 부탁드려요 드라마 사설이 넘길어 ...,Sadness


In [25]:
test=pd.read_csv("C:\\Users\\Fossati\\Desktop\\Tesi\\Dati\\Dati_FineTuningBERT\\Train_Test_emotion\\Strategy_fin\\test_def.csv")
test

,Unnamed: 0,level_0,Sentence,Emotions
0,0,33476,This does not sound like a good situation for ...,Anger
1,1,11870,There's really no way to know since my lady ho...,Neutral
2,2,34327,Considering that cheating is a mark of high-st...,Anger
3,3,50190,i am feeling a little apprehensive but i m sur...,Fear
4,4,49266,i still feel like there are more than enough t...,Joy
...,...,...,...,...
9484,9484,92647,"멋있을 수도 있는 게 하나도 안 멋있고, 감동적일 수도 있는 장면에 아무런 감동도 ...",Surprise
9485,9485,92038,배우가 너무 아깝다고 생각했다.,Surprise
9486,9486,65497,음 기왕 일산 까지 보신 다면 렉서스 의 suv 하브 모델 은 어떠하실 지요 . R...,Neutral
9487,9487,45413,발연기. 득보잡 배우들. 장르는 저예산 에로.,Fear


In [6]:
#creiamo le liste di train e test. 1.8k train 250 test  
train_texts=train['Sentence']
train_labels=train['Emotions']
test_texts=test['Sentence']
test_labels=test['Emotions']


<br><br>

## **Split the data into training and test sets**

In [9]:
len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(80667, 80667, 9489, 9489)

Here's an example of a training label and review:

In [10]:
train_labels[0], train_texts[0]

('Anger',
 'i have to admit that i feel a little irate as well but its under control')

<br><br>

## **Encode data for BERT**

We're going to transform our texts and labels into a format that BERT (via Huggingface and PyTorch) will understand. This is called *encoding* the data.

Here are the steps we need to follow:

1. The labels&mdash;in this case, Goodreads genres&mdash;need to be turned into integers rather than strings.

2. The texts&mdash;in this case, Goodreads reviews&mdash;need to be truncated if they're more than 512 tokens or padded if they're fewer than 512 tokens. The tokens, or words in the texts, also need to be separated into "word pieces" and matched to their embedding vectors.

3. We need to add special tokens to help BERT:

| BERT special token | Explanation |
| --------------| ---------|
| [CLS] | Start token of every document. |
| [SEP] | Separator between each sentence |
| [PAD] | Padding at the end of the document as many times as necessary, up to 512 tokens |
|  &#35;&#35; | Start of a "word piece" |




In [11]:
tokenizer = XLMRobertaTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Here we will create a map of our labels, or Goodreads genres, to integer keys. We take the unique labels, and then we make a dictionary that associates each label/tag with an integer.

**Note:** HuggingFace documentation sometimes refers to "labels" as "tags" but these are the same thing. We use "labels" throughout this notebook for clarity.

In [12]:
#unique_labels = set(label for label in train_labels)
#label2id = {label: id for id, label in enumerate(unique_labels)}
#id2label = {id: label for label, id in label2id.items()}

In [13]:
unique_labels={'Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger','Fear'}
label2id={'Sadness':0, 'Surprise':1, 'Joy':2, 'Neutral':3, 'Anger':4, 'Fear':5}
id2label={ 0:'Sadness', 1:'Surprise', 2:'Joy', 3:'Neutral', 4:'Anger', 5:'Fear'} 

In [14]:
unique_labels

{'Anger', 'Fear', 'Joy', 'Neutral', 'Sadness', 'Surprise'}

In [15]:
label2id.keys()

dict_keys(['Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger', 'Fear'])

In [16]:
id2label.keys()

dict_keys([0, 1, 2, 3, 4, 5])

Now let's encode our texts and labels!

In [17]:
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
test_encodings  = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=max_length)

train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]

**Examine a Goodreads review in the training set after encoding**

In [18]:
' '.join(train_encodings[29].tokens[0:100])

"<s> ▁Got ▁it ! ▁I ▁wasn ' t ▁aware ▁that ▁it ▁was ▁a ▁slu r , ▁and ▁thought ▁that ▁it ▁was ▁just ▁a ▁general ▁term ▁used . ▁Thanks ▁for ▁the ▁info . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>"

**Examine a Goodreads review in the test set after encoding**

In [19]:
' '.join(test_encodings[0].tokens[0:100])

'<s> ▁This ▁does ▁not ▁sound ▁like ▁a ▁good ▁situation ▁for ▁a ▁child ▁at ▁all ▁ imo . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

**Examine the training labels after encoding**

In [20]:
set(train_labels_encoded)

{0, 1, 2, 3, 4, 5}

**Examine the test labels after encoding**

In [21]:
set(test_labels_encoded)

{0, 1, 2, 3, 4, 5}

<br><br>

## **Make a custom Torch dataset**

Here we combine the encoded labels and texts into dataset objects. We use the custom Torch `MyDataSet` class to make a `train_dataset` object from  the `train_encodings` and `train_labels_encoded`. We also make a `test_dataset` object from `test_encodings`, and `test_labels_encoded`.

In [22]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [23]:
train_dataset = MyDataset(train_encodings, train_labels_encoded)
test_dataset = MyDataset(test_encodings, test_labels_encoded)

In [24]:
' '.join(train_dataset.encodings[0].tokens[0:100])

'<s> ▁i ▁have ▁to ▁admit ▁that ▁i ▁feel ▁a ▁little ▁i rate ▁as ▁well ▁but ▁its ▁under ▁control </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

In [25]:
' '.join(test_dataset.encodings[1].tokens[0:100])

"<s> ▁There ' s ▁really ▁no ▁way ▁to ▁know ▁since ▁my ▁lady ▁hormone s ▁prevent ▁me ▁from ▁remember ing ▁sexual ▁a ssa ul ters ' ▁identi ties . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>"

<br><br>

## **Load pre-trained BERT model**

In [26]:
# The model_name needs to match the name used for the tokenizer above.
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label)).to(device_name)

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

<br><br>

## **Set the BERT fine-tuning parameters**

These are the arguments we'll set in the HuggingFace TrainingArguments objects, which we'll then pass to the HuggingFace Trainer object. There are many more possible arguments, but here we highlight the basics and some common gotchas.

When training your own model, you should search over these parameters to find the best settings for your particular dataset. You should use a held-out set of validation data for this step.

| Parameter | Explanation |
|-----------| ------------|
| num_train_epochs | total number of training epochs (how many times to pass through the entire dataset; too much can cause overfitting) |
| per_device_train_batch_size | batch size per device during training |
| per_device_eval_batch_size |  batch size for evaluation |
|  warmup_steps |  number of warmup steps for learning rate scheduler (set lower because of small dataset size) |
| weight_decay | strength of weight decay (reduces size of weights, like regularization) |
| output_dir | output directory for the fine-tuned model and configuration files |
| logging_dir | directory for storing logs |
| logging_steps | how often to print logging output (so that we can stop training early if the loss isn't going down) |
| evaluation_strategy | evaluate while training so that we can see the accuracy going up |

In [27]:
training_args=TrainingArguments(
    num_train_epochs=3.75,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=8000,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
    save_strategy='no'
)

<br><br>

## **Fine-tune the BERT model**

First, we define a custom evaluation function that returns the accuracy. You could modify this function to return precision, recall, F1, and/or other metrics.

In [28]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

Then we create a HuggingFace `Trainer` object using the `TrainingArguments` object that we created above. We also send our `compute_metrics` function to the `Trainer` object, along with our test and train datasets.

**Note:** This is what we've been aiming for this whole time! All the work of tokenizing, creating datasets, and setting the training arguments was for this cell.

In [29]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset (usually a validation set; here we just send our test set)
    compute_metrics=compute_metrics      # our custom evaluation function 
)

Time to finally fine-tune! 

Be patient; if you've set everything in Colab to use GPUs, then it should only take a minute or two to run, but if you're running on CPU, it can take hours.

After every 10 steps (as we specified in the TrainingArguments object), the trainer will output the current state of the model, including the training loss, validation ("test") loss, and accuracy (from our `compute_metrics` function).

You should see the loss going down and the accuracy going up. If instead they are staying the same or oscillating, you probably need to change the fine-tuning parameters.

In [30]:
trainer.train()                                                                                                                

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 80667
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 18908


Step,Training Loss,Validation Loss,Accuracy
8000,0.995200,0.728645,0.727685
16000,0.688100,0.550293,0.800927


***** Running Evaluation *****
  Num examples = 9489
  Batch size = 16
***** Running Evaluation *****
  Num examples = 9489
  Batch size = 16


Step,Training Loss,Validation Loss,Accuracy
8000,0.995200,0.728645,0.727685
16000,0.688100,0.550293,0.800927




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=18908, training_loss=0.7994720672032837, metrics={'train_runtime': 8086.6523, 'train_samples_per_second': 37.407, 'train_steps_per_second': 2.338, 'total_flos': 1.9899343319560704e+16, 'train_loss': 0.7994720672032837, 'epoch': 3.75})

In [ ]:
#tempo di training: 4 ore

<br><br>

## **Save fine-tuned model**

The following cell will save the model and its configuration files to a directory in Colab. To preserve this model for future use, you should download the model to your computer.

In [31]:
trainer.save_model(cached_model_directory_name)

Saving model checkpoint to Emotion_fin_xlmr
Configuration saved in Emotion_fin_xlmr/config.json
Model weights saved in Emotion_fin_xlmr/pytorch_model.bin


(Optional) If you've already fine-tuned and saved the model, you can reload it using the following line. You don't have to run fine-tuning every time you want to evaluate.

<br><br>

## **Evaluate fine-tuned model**

The following function of the `Trainer` object will run the built-in evaluation, including our `compute_metrics` function.

In [32]:
trainer.evaluate() 

***** Running Evaluation *****
  Num examples = 9489
  Batch size = 16


{'epoch': 3.75,
 'eval_accuracy': 0.8127305300874696,
 'eval_loss': 0.5225909352302551,
 'eval_runtime': 65.5494,
 'eval_samples_per_second': 144.761,
 'eval_steps_per_second': 9.062}

But we might want to do more fine-grained analysis of the model, so we extract the predicted labels.

In [33]:
predicted_results = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 9489
  Batch size = 16


In [34]:
predicted_results.predictions.shape

(9489, 6)

In [35]:
predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

In [36]:
len(predicted_labels)

9489

In [37]:
print(classification_report(test_labels, 
                            predicted_labels))     

              precision    recall  f1-score   support

       Anger       0.80      0.83      0.81      1547
        Fear       0.80      0.85      0.83      1497
         Joy       0.82      0.81      0.81      1606
     Neutral       0.88      0.79      0.83      1910
     Sadness       0.83      0.81      0.82      1457
    Surprise       0.74      0.79      0.76      1472

    accuracy                           0.81      9489
   macro avg       0.81      0.81      0.81      9489
weighted avg       0.81      0.81      0.81      9489



In [38]:
#la novità è che c'è risultato generale sufficientemente buoni per tutte e 5 classi

In [ ]:
#RELOAD THE MODEL

In [ ]:
#valutazione per domain

In [26]:
def prediction_test(dataset):  #dataset fa riferimento al set di dati che vogliamo passare, modello dipende dal tipo di FT
#poi ci sarà anche la funzione per predire senza labels
    tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
    modello=XLMRobertaForSequenceClassification.from_pretrained(cached_model_directory_name)
    test_texts=dataset['Sentence']
    test_labels=dataset['Emotions']

    #unique_labels = set(label for label in test_labels)
    #label2id = {label: id for id, label in enumerate(unique_labels)}
    #id2label = {id: label for label, id in label2id.items()}
    unique_labels={'Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger','Fear'}
    label2id={'Sadness':0, 'Surprise':1, 'Joy':2, 'Neutral':3, 'Anger':4, 'Fear':5}
    id2label={ 0:'Sadness', 1:'Surprise', 2:'Joy', 3:'Neutral', 4:'Anger', 5:'Fear'}  

#train_encodings = tok(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
    test_encodings  = tok(test_texts.tolist(), truncation=True, padding=True, max_length=max_length)

#train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
    test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]
    

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

#train_dataset = MyDataset(train_encodings, train_labels_encoded)
    test_dataset = MyDataset(test_encodings, test_labels_encoded)
    
    training_args = TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1500,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps'     # evaluate during fine-tuning so that we can see progress
)
    
    trainer = Trainer(model=modello, args=training_args)  #basta avere il modello come parametro

    predicted_results=trainer.predict(test_dataset)
    
    predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
    predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
    predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

#len(predicted_labels)

    return print(classification_report(test_labels,predicted_labels)), print(confusion_matrix(test_labels,predicted_labels,labels=['Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger','Fear']))

In [27]:
prediction_test(test)

loading file https://huggingface.co/xlm-roberta-base/resolve/main/sentencepiece.bpe.model from cache at C:\Users\Fossati/.cache\huggingface\transformers\9df9ae4442348b73950203b63d1b8ed2d18eba68921872aee0c3a9d05b9673c6.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
loading file https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\daeda8d936162ca65fe6dd158ecce1d8cb56c17d89b78ab86be1558eaef1d76a.a984cf52fc87644bd4a2165f1e07e0ac880272c1e82d648b4674907056912bd7
loading file https://huggingface.co/xlm-roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/xlm-roberta-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at C:\Users\Fossati/.

              precision    recall  f1-score   support

       Anger       0.82      0.84      0.83      1547
        Fear       0.82      0.87      0.85      1497
         Joy       0.83      0.83      0.83      1606
     Neutral       0.89      0.81      0.85      1910
     Sadness       0.84      0.83      0.83      1457
    Surprise       0.75      0.79      0.77      1472

    accuracy                           0.83      9489
   macro avg       0.83      0.83      0.83      9489
weighted avg       0.83      0.83      0.83      9489

[[1213   45   40   18   68   73]
 [  43 1165   51   62   42  109]
 [  55   70 1328   61   48   44]
 [  32  145   96 1545   87    5]
 [  58   70   45   32 1293   49]
 [  50   63   42    9   32 1301]]


(None, None)

In [ ]:
#{'Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger','Fear'}

#le classi vanno lette in quest ordine

#surprise verso fear se errore
#neutral verso surprise in caso di errore
#surprise scelto in generale maggiormente come ripiego in caso di errore, tranne che per sadness

In [28]:
prediction_test(test[:2972])   #tabella per inglesi    #0.86

loading file https://huggingface.co/xlm-roberta-base/resolve/main/sentencepiece.bpe.model from cache at C:\Users\Fossati/.cache\huggingface\transformers\9df9ae4442348b73950203b63d1b8ed2d18eba68921872aee0c3a9d05b9673c6.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
loading file https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\daeda8d936162ca65fe6dd158ecce1d8cb56c17d89b78ab86be1558eaef1d76a.a984cf52fc87644bd4a2165f1e07e0ac880272c1e82d648b4674907056912bd7
loading file https://huggingface.co/xlm-roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/xlm-roberta-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at C:\Users\Fossati/.

              precision    recall  f1-score   support

       Anger       0.88      0.90      0.89       526
        Fear       0.90      0.95      0.92       330
         Joy       0.91      0.91      0.91       552
     Neutral       0.88      0.75      0.81       599
     Sadness       0.93      0.96      0.94       471
    Surprise       0.82      0.88      0.85       494

    accuracy                           0.88      2972
   macro avg       0.89      0.89      0.89      2972
weighted avg       0.88      0.88      0.88      2972

[[450   4   0   2   9   6]
 [  4 437   9  20   6  18]
 [  4  12 505  22   7   2]
 [ 13  64  36 447  38   1]
 [  9  11   7  15 476   8]
 [  5   4   1   4   4 312]]


(None, None)

In [18]:
#{'Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger','Fear'}

#per l'inglese ci siamo, otteniamo ottimi risultati
#unico pattern per neutral, che in caso di errore viene classificato maggiormente come surprise

In [29]:
prediction_test(test[2972:5981])   

loading file https://huggingface.co/xlm-roberta-base/resolve/main/sentencepiece.bpe.model from cache at C:\Users\Fossati/.cache\huggingface\transformers\9df9ae4442348b73950203b63d1b8ed2d18eba68921872aee0c3a9d05b9673c6.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
loading file https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\daeda8d936162ca65fe6dd158ecce1d8cb56c17d89b78ab86be1558eaef1d76a.a984cf52fc87644bd4a2165f1e07e0ac880272c1e82d648b4674907056912bd7
loading file https://huggingface.co/xlm-roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/xlm-roberta-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at C:\Users\Fossati/.

              precision    recall  f1-score   support

       Anger       0.83      0.88      0.86       537
        Fear       0.91      0.93      0.92       312
         Joy       0.85      0.86      0.86       561
     Neutral       0.78      0.66      0.72       604
     Sadness       0.89      0.92      0.90       499
    Surprise       0.76      0.81      0.79       496

    accuracy                           0.83      3009
   macro avg       0.84      0.84      0.84      3009
weighted avg       0.83      0.83      0.83      3009

[[457   8   1  14  13   6]
 [ 10 404  15  40  15  12]
 [  5  14 484  39  15   4]
 [ 16  78  60 398  48   4]
 [ 14  21   8  16 475   3]
 [  9   4   0   2   6 291]]


(None, None)

In [ ]:
#{'Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger','Fear'}

#surprise verso neutral
#joy verso neutral
#neutral verso surprise
#

In [30]:
prediction_test(test[5981:])   #tabella per korean   


loading file https://huggingface.co/xlm-roberta-base/resolve/main/sentencepiece.bpe.model from cache at C:\Users\Fossati/.cache\huggingface\transformers\9df9ae4442348b73950203b63d1b8ed2d18eba68921872aee0c3a9d05b9673c6.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
loading file https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\daeda8d936162ca65fe6dd158ecce1d8cb56c17d89b78ab86be1558eaef1d76a.a984cf52fc87644bd4a2165f1e07e0ac880272c1e82d648b4674907056912bd7
loading file https://huggingface.co/xlm-roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/xlm-roberta-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at C:\Users\Fossati/.

              precision    recall  f1-score   support

       Anger       0.75      0.71      0.73       484
        Fear       0.76      0.82      0.79       855
         Joy       0.71      0.69      0.70       493
     Neutral       0.99      0.99      0.99       707
     Sadness       0.67      0.63      0.65       487
    Surprise       0.65      0.67      0.66       482

    accuracy                           0.77      3508
   macro avg       0.76      0.75      0.75      3508
weighted avg       0.77      0.77      0.77      3508

[[306  33  39   2  46  61]
 [ 29 324  27   2  21  79]
 [ 46  44 339   0  26  38]
 [  3   3   0 700   1   0]
 [ 35  38  30   1 342  38]
 [ 36  55  41   3  22 698]]


(None, None)

In [ ]:
#{'Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger','Fear'}

#sadness verso fear
#surprise verso fear
#performance importante su neutral, che per giunta non viene mai scelta come ripiego in caso di errore da altre classificazioni
#fear tende ad andare verso surprise se ci sono errori

In [ ]:
#questo modello è buono per emotion, il migliore che abbiamo a livello di risultati